# 1\. Включение GPU и проверка



In [1]:
!nvidia-smi

Mon Aug  4 12:43:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             29W /   70W |   12734MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 2\. Установка системной зависимости для PyAudio

In [2]:
!apt-get update
!apt-get install -y portaudio19-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

# 3\. Удаление существующей директории Mini-Omni

In [3]:
!rm -rf mini-omni

# 4\. Клонирование репозитория и переход в него

In [4]:
!git clone https://github.com/gpt-omni/mini-omni.git
%cd mini-omni

Cloning into 'mini-omni'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 124 (delta 29), reused 13 (delta 10), pack-reused 84 (from 1)
Receiving objects: 100% (124/124), 12.21 MiB | 12.49 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/mini-omni


# 5\. Установка основных Python-зависимостей

In [5]:
!pip install -r requirements.txt --break-system-packages
!pip install tokenizers transformers gradio gradio-client --upgrade --break-system-packages

  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached gradio-4.42.0-py3-none-any.whl.metadata (15 kB)
  Using cached gradio_client-1.3.0-py3-none-any.whl.metadata (7.1 kB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Using cached gradio-4.42.0-py3-none-any.whl (16.8 MB)
Using cached gradio_client-1.3.0-py3-none-any.whl (318 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.11.0
    Uninstalling gradio_client-1.11.0:
      Successfully uninstalled gradio_client-1.11.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.39.0
    Uninstalling gradio-5.39.0:
      Successfully uninstalled gradio-5.39.0
ERROR: pip's dependency resolver doe

Останавливаем все фоновые процессы `server.py`

In [6]:
!kill $(ps aux | grep 'python server.py' | awk '{print $2}')

^C


# 6\. Запуск сервера в фоновом режиме

In [7]:
!nohup python server.py --ip '0.0.0.0' --port 60807 > server.log 2>&1 &

Проверь логи сервера:

3\. **Проверь, слушает ли порт 60807**

In [8]:
!lsof -i :60807

COMMAND     PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
pt_main_t 41876 root   56u  IPv4 845938      0t0  TCP *:60807 (LISTEN)


# новый подход

Установи PyNgrok и добавь свой Auth Token

In [9]:
!pip install pyngrok --break-system-packages
!ngrok config add-authtoken youre_token # Замени на свой токен

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# 4\. **Измени `webui/omni_gradio.py` для отключения `share`

In [10]:
!sed -i 's/share=True/share=False/' webui/omni_gradio.py

# **Запусти Ngrok-туннель и Gradio**

In [11]:

from pyngrok import ngrok

# Закрываем любые предыдущие туннели ngrok, если они были
ngrok.kill()

# Создаем туннель для порта Gradio (7860)
public_url = ngrok.connect(7860)
print(f"Ngrok Public URL for Gradio: {public_url}")

# Устанавливаем API_URL для Gradio (важно, чтобы он совпадал с тем, на котором слушает server.py)
%env API_URL=http://0.0.0.0:60807/chat

# Запускаем Gradio БЕЗ флага --share, так как мы изменили его в файле
!python webui/omni_gradio.py


Ngrok Public URL for Gradio: NgrokTunnel: "https://a1d798b75ab3.ngrok-free.app" -> "http://localhost:7860"
env: API_URL=http://0.0.0.0:60807/chat


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
filepath: /tmp/gradio/c245bcc1689f06626e60cae2ea855c790bb6ba907e5a983715f01122964f8982/audio.wav
first chunk time cost: 1.832
filepath: None
Created dataset file at: .gradio/flagged/dataset1.csv
filepath: /tmp/gradio/6760e8930d545ffbdf0c085d6eb908507816f75df8c5aee938dc9bf393f80a80/audio.wav
first chunk time cost: 0.835
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3113, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/mini-omni/webui/omni_gradio.py", line 82, in <module>
    fire.Fire(main)
  File "/usr/local/lib/python3.11/dist-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, n

In [8]:
!cat server.log

checkpoint directory ./checkpoint not found, downloading from huggingface
Fetching 7 files:  14%|█▍        | 1/7 [00:00<00:02,  2.79it/s]

# 8\. Запуск Gradio Demo


In [10]:
%env API_URL=http://0.0.0.0:60807/chat
!python webui/omni_gradio.py --share

env: API_URL=http://0.0.0.0:60807/chat
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3113, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/mini-omni/webui/omni_gradio.py", line 82, in <module>
    fire.Fire(main)
  File "/usr/local/lib/python3.11/dist-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
                                ^^^^^^^^^^^^^^^^^^^^
  File "/us